In [1]:
import pandas as pd
import datetime
import numpy as np

# Читаем данные.
try:
  ioka_report = pd.read_excel('ioka\'s_report.xlsx')
  bank_report = pd.read_excel('bank\'s_report.xlsx')
except Exception as e:
  print(f"Ошибка при чтении файлов: {e}")
  exit()

# Привести формат даты в 'bank\'s_report.xlsx' к формату 'ioka\'s_report.xlsx'
bank_report['Дата операции'] = pd.to_datetime(bank_report['Дата операции'], format='%d.%m.%Y %H:%M:%S')

# Функции для дальнейшего пользования.
def convert_to_float(val):
    return float(val)
def convert_payment_system(val):
    new_val = str(val)
    return new_val.upper()
def convert_to_abs(val):
    new_val = np.abs(val)
    return new_val
def convert_to_str(val):
    return str(val)

# Необходимые для работы преобразования.

# Заменяем запятые чисел в точки. Чтобы стали идентичны в обеих файлах.
ioka_report['Сумма'] = ioka_report['Сумма'].astype(str).str.replace(',', '.')
ioka_report['Комиссия'] = ioka_report['Комиссия'].astype(str).str.replace(',', '.')
ioka_report['Выплата'] = ioka_report['Выплата'].astype(str).str.replace(',', '.')

# Конвертируем столбы. Чтобы типы данных стали идентичны.
ioka_report['Сумма']=ioka_report['Сумма'].apply(convert_to_float)
ioka_report['Комиссия']=ioka_report['Комиссия'].apply(convert_to_float)
ioka_report['Выплата']=ioka_report['Выплата'].apply(convert_to_float)
# Избавляемся от миллисекунд в данных.
ioka_report['Дата и время'] = ioka_report['Дата и время'].dt.round('S')
# Также приводим в один тип данных.
bank_report['Платежная система']=bank_report['Платежная система'].apply(convert_payment_system)
bank_report['Дата операции']=pd.to_datetime(bank_report['Дата операции'])
bank_report['Комиссия']=bank_report['Комиссия'].apply(convert_to_abs)

# Переименовываем столбы в обеих файлах чтобы были одинаковые названия столбов. Выручит в джойнах.
ioka_report.rename(columns={'Дата и время':'Дата операции'}, inplace=True)
bank_report.rename(columns={'Всего':'Выплата'}, inplace=True)

# Снижаем секунды до ближайшей минуты во времени. Чтобы избежать конфликтов в секундах.
ioka_report['Дата операции'] = ioka_report['Дата операции'].dt.floor('min')
bank_report['Дата операции'] = bank_report['Дата операции'].dt.floor('min')
# На этом ранние манипуляции с данными были закончены.


# Находим совпадающие платежи по дате, RRN, сумме платежа и комиссии.
# Также следует добавить можно было бы добавить к требованию задания сверку и по платежной системе. Чтобы была точность в 100%.
second_merged_df = pd.merge(bank_report, ioka_report, how='inner', on=['Дата операции', 'RRN', 'Сумма', 'Комиссия', 'Выплата'])


# Если платеж не найден в одном из отчетов, то можно добавить его в список расхождений как не найденный в обеих файлах.
# Если сумма или комиссия различаются, также добавьте их в список расхождений.

# Алгоритм для определения расхождений:
difference = [] # Список для хранения здесь расхождения.
for index, row in ioka_report.iterrows(): # Циклом пробегаем по каждой строке.
  match = bank_report[(bank_report['Дата операции'] == row['Дата операции']) & (bank_report['RRN'] == row['RRN'])] # Ищем по дате и RRN в данных ioka и эквайера.
  if len(match) == 0: # Если найдено лишь в данных ioka и не нашлось в данных эквайера, записываем в наш список.
    difference.append({'Дата операции': row['Дата операции'], 'RRN': row['RRN'], 'Причина': 'Отсутствует в банковском отчете'})
  elif len(match) == 1: # Если найдено в обеих данных, начинаем поиск в расхождении в сумме либо комиссии.
    bank_row = match.iloc[0] # Извлекаем данные из первой совпавшей по дате и RRN. Чтобы теперь сверить по комиссии и сумме.
    if row['Сумма'] != bank_row['Сумма'] or row['Комиссия'] != bank_row['Комиссия']: # Находим ошибку по комиссии или сумме.
      difference.append({'Дата операции': row['Дата операции'], 'RRN': row['RRN'], 'Причина': 'Различие в сумме или комиссии'}) # Сохраняем в наш список.

# Образуем из списка в данные.
difference = pd.DataFrame(difference)

# Моя небольшая пометка для себя.
# Выясняется что у некоторых данных различие во времени на 2 секунды.
# В bank_report это 22.02.29 который соответственно округленный в 22.02
# В ioka_report это 22.02.31 округленный в 22.03
# Значит возможно мы также теряем и некоторое количество в совпавшихся платежах из second_merged_df
# Решение есть с тем чтобы округлить всё до ближайшего часа. И смотреть не будут ли такие же проблемы с 31 минутой и 29-ой.
# Пример 22.31 в 23.00, а 22.29 в 22.00. Это портит всё, я нуждаюсь в решений руководителей этой проблемы.
# Обновление, в решений этой проблемы нам может помочь округление с понижением все в минуте. Но стоит помнить, что все равно будут исключения.
# Пример 22.02.29 в 22.02, а 22.02.31 также в 22.02. Но 22:40:59 станет 22:40:00 а 22:41:00 будет оставаться 22:41:00. И они будут в обеих файлах.


# Конвертируем RRN из чисел в строку. Без этого excel отказывается правильно отображать наши данные.
second_merged_df['RRN'] = second_merged_df['RRN'].apply(convert_to_str)
difference['RRN'] = difference['RRN'].apply(convert_to_str)

with pd.ExcelWriter('результат.xlsx') as writer:
  second_merged_df.to_excel(writer, sheet_name='Совпавшие платежи', index=False)
  difference.to_excel(writer, sheet_name='Расхождения и причины', index=False)

# После скачивания настоятельно рекомендую в ручную настроить ширину столбов в excel. Для того чтобы увидеть данные.